In [1]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os
import time

In [2]:
fixPrompt = "I'm a ReactJS developer. I need to convert the following information into a raw string format for use in my React app. Finally, provide me the raw text which I can directly display on my React app. So, provide a single line raw text in string format. Follow these instructions:1. Do not modify any words in the text.2. Add newlines (\n) for better readability.3. Add double asterisks (**) to make important titles bold. Only apply this to existing titles and headings in the text; do not forcefully create new ones.4. Every topic which has a sub-topic in it should be bold.5. If sub-topics are further nested, then bold them too.6. If numbering is needed, then give numbering to sub-concepts.7. Any word followed by a colon (:) should be bold.8. Do not add any quotes (single or double) around the text.9. Do not add any prefixes like `const rawtext =`."

In [3]:
def chatgptResponse(client,prompt,row_idx,col):
    new_text = ""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            stream=True,
        )
        for chunk in response:
            if chunk.choices[0].delta.content is not None:
                new_text += f'{chunk.choices[0].delta.content}'
        return new_text
    except Exception as e:
        print(f"\n\nError updating row {row_idx}, column '{col}': {e}")

In [4]:

def update_cell(df, col, row_idx, client):
    original_text = df.at[row_idx, col]
    prompt = f"{fixPrompt} {original_text}"
    new_text = chatgptResponse(client,prompt,row_idx,col)
    # # Ensure thread-safe update
    # with lock:
    df.at[row_idx, col] = new_text
    print(f"Row {row_idx}, column '{col}' updated",end=" ")

In [5]:
def update_column(df,row,col,client):
    for row_idx in row:
        # print(row_idx)
        if row_idx%20 == 0:
            time.sleep(30)
        update_cell(df, col, row_idx,client)
        df[col] = df[col].apply(lambda x: x.replace('\n', '\\n') if isinstance(x, str) else x)

In [6]:
def main(file_path, output_file_path, client):
    df = pd.read_excel(file_path)
    df = df.dropna()
    threads = []
    # row = [r for r in df.index]
    columns = [col for col in df.columns]
    columns = columns[5:]
    # row = [0,1,5,6,8,9,12,13,15,18,19,20,21,22,23,26,27,28,29,30,33,34,35,36,39,40,41,42,43,46,50,51,53,54,55,56,57,58,60,61,62,64,65,66,67,68,69]
    row = [r for r in df.index]
    # row = row[0:9]
    
    for col in columns:
        update_column(df,row,col,client)
        print(f"\nComplete {col} updated")
    #     thread = Thread(update_column, args=(df,row, col, client))
    #     threads.append(thread)
    #     thread.start()
    
    # for thread in threads:
    #     thread.join()
    
    # Use xlsxwriter to write the DataFrame to Excel
    writer = pd.ExcelWriter(output_file_path, engine='xlsxwriter')
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    writer.close()
    print("\n\nProcessing complete and output file saved")

In [7]:
if __name__ == "__main__":
    api_key = os.getenv('OPEN_API_KEY')
    client = OpenAI(api_key=api_key)
    file_path = 'ltass.xlsx' 
    output_file_path = 'verifiedss.xlsx' 
    main(file_path, output_file_path,client)

Row 1, column 'Summary' updated Row 2, column 'Summary' updated Row 3, column 'Summary' updated Row 4, column 'Summary' updated Row 5, column 'Summary' updated Row 6, column 'Summary' updated Row 8, column 'Summary' updated Row 9, column 'Summary' updated Row 10, column 'Summary' updated Row 11, column 'Summary' updated Row 12, column 'Summary' updated Row 13, column 'Summary' updated Row 15, column 'Summary' updated Row 16, column 'Summary' updated Row 17, column 'Summary' updated Row 18, column 'Summary' updated Row 19, column 'Summary' updated Row 20, column 'Summary' updated Row 22, column 'Summary' updated Row 23, column 'Summary' updated Row 24, column 'Summary' updated Row 25, column 'Summary' updated Row 26, column 'Summary' updated Row 27, column 'Summary' updated Row 29, column 'Summary' updated Row 30, column 'Summary' updated Row 31, column 'Summary' updated Row 32, column 'Summary' updated Row 33, column 'Summary' updated Row 34, column 'Summary' updated Row 36, column 'Su